In [63]:

url = "https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N"
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

Members = soup.findAll(class_ = 'Members--bio u-richtext')
state_district = soup.findAll(class_ = 'Hero-title')
Money = soup.findAll(class_ = 'Members--number')


In [2]:
def get_candidate_names(Members):
    members_list = []
    member_affiliation = []

    for i in range(len(Members)):
        if i == 0:
            name_affiliation = Members[0].text
            members_list.append(
                re.split(r"[\t\n()]", name_affiliation)[2].strip()
            )
            member_affiliation.append(
                re.split(r"[\t\n()]", name_affiliation)[3].strip()
            )
        else:
            
            name_affiliation = Members[i].text
            members_list.append(
                re.split(r"[\t()]", name_affiliation)[10].strip()
                )
            member_affiliation.append(
                re.split(r"[\t()]", name_affiliation)[11].strip()
                )
    return members_list, member_affiliation


In [3]:
def get_incumbent(Members):    
    list = []
    for th in Members:
     #print(th.text.strip())
        list.append(re.split(r"[\t\n()]", th.text))
    incumbent_list = []
    for i in range(len(list)):
        if any("Incumbent" in item for item in list[i]):
            incumbent_list.append(1)
        else:
            incumbent_list.append(0)
    return incumbent_list

In [4]:
def get_percentage(Members):
    list = []
    for th in Members:
     #print(th.text.strip())
        list.append(re.split(r"[()]", th.text))
    incumbent_list = []
    for i in range(len(list)):
        incumbent_list.append(
            list[i][3]
            )
    return incumbent_list

In [5]:
def get_winner(Members):
    percent_num = []
    for i in range(len(percentage_list)):
        percent_num.append(float(percentage_list[i].split("%")[0]))
    
    winner_list = []
    for i in range(len(percent_num)):
        if percent_num[i] == max(percent_num):
            winner_list.append(1)
        else:
            winner_list.append(0)
    return winner_list




In [51]:
def get_state_district(state_district):
    state_list = []
    district_list = []
    for i in range(len(Members)):
        state_list.append(state_district[0].text.split()[0])
        district_list.append(state_district[0].text.split()[2])

    return state_list, district_list

In [56]:
incumbent_list = get_incumbent(Members)
percentage_list = get_percentage(Members)
winner_list = get_winner(Members)
candidate_list = get_candidate_names(Members)
state_list = get_state_district(state_district)[0]
district_list = get_state_district(state_district)[1]

In [53]:
state_list

['Tennessee', 'Tennessee', 'Tennessee', 'Tennessee']

In [61]:
candidate_df = pd.DataFrame({"Candidate Name" : candidate_list[0],
                             "Affiliation" : candidate_list[1],
                             "Incumbent": incumbent_list,
                             "Percentage of Vote" : percentage_list,
                             "Winner" : winner_list,
                             "State": state_list,
                             "District" : district_list}
                             )

In [62]:
candidate_df

,Candidate Name,Affiliation,Incumbent,Percentage of Vote,Winner,State,District
0,Mark Green,R,1,69.9% of vote,1,Tennessee,07
1,Kiran Sreepada,D,0,27.3% of vote,0,Tennessee,07
2,Ronald Brown,I,0,2.2% of vote,0,Tennessee,07
3,Scott Vieira Jr,I,0,0.6% of vote,0,Tennessee,07


In [67]:
len(Money)

12

In [68]:
Money[0].text

'$1,194,960'